In [2]:
# Data Manipulation
import numpy as np
import pandas as pd

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Analysis
from sklearn.metrics import mean_absolute_error
import scipy as sp
import statsmodels as sm
from scipy.stats import wilcoxon, mannwhitneyu

In [3]:
df_tf_input = pd.read_csv("data-collection/outputs/performance-data-true-false.csv")
df_ft_input = pd.read_csv("data-collection/outputs/performance-data-false-true.csv")
df_ff_input = pd.read_csv("data-collection/outputs/performance-data-false-false.csv")

df_gt_input = pd.read_csv("data-collection/groundtruths/target-data.csv")

### Compare number of corners found

In [3]:
# Add column to each DF saying how many corners are not Nan
df_tf = df_tf_input[["filename", "corner-1", "corner-2", "corner-3", "corner-4", "corner-5"]]
df_ff = df_ff_input[["filename", "corner-1", "corner-2", "corner-3", "corner-4", "corner-5"]]
df_gt = df_gt_input[["filename", "corner-1", "corner-2", "corner-3", "corner-4", "corner-5"]]

df_tf = df_tf.replace('FAILED', np.nan, regex=True)
df_ff = df_ff.replace('FAILED', np.nan, regex=True)

df_tf["tf-num-corners"] = df_tf.count(axis=1)-1
df_ff["ff-num-corners"] = df_ff.count(axis=1)-1
df_gt["gt-num-corners"] = df_gt.count(axis=1)-1

c:\Users\Kim\anaconda3\envs\analysis\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [4]:
# Remove all other columns but name and corner num
df_tf = df_tf[["filename", "tf-num-corners"]]
df_ff = df_ff[["filename", "ff-num-corners"]]
df_gt = df_gt[["filename", "gt-num-corners"]]

In [5]:
# Merge on filename
corner_nums = df_gt.merge(df_tf, on="filename").merge(df_ff, on="filename")

In [6]:
# Get MAE compared to GTs
error_tf = mean_absolute_error(corner_nums["gt-num-corners"], corner_nums["tf-num-corners"])
error_ff = mean_absolute_error(corner_nums["gt-num-corners"], corner_nums["ff-num-corners"])

In [7]:
print(error_tf)
print(error_ff)

0.8309859154929577
1.1549295774647887


In [8]:
tf_error_dist = np.abs(corner_nums["gt-num-corners"]-corner_nums["tf-num-corners"])
ff_error_dist = np.abs(corner_nums["gt-num-corners"]-corner_nums["ff-num-corners"])

res = wilcoxon(tf_error_dist, ff_error_dist)

print(res)

WilcoxonResult(statistic=48.0, pvalue=0.0002451354174095733)


### Compare error of corner points

In [37]:
df_tf = df_tf_input[["filename", "height", "width", "corner-1", "corner-2", "corner-3", "corner-4", "corner-5"]]
df_ff = df_ff_input[["filename", "height", "width", "corner-1", "corner-2", "corner-3", "corner-4", "corner-5"]]
df_gt = df_gt_input[["filename", "height", "width", "corner-1", "corner-2", "corner-3", "corner-4", "corner-5"]]

df_tf = df_tf.replace('FAILED', np.nan, regex=True)
df_ff = df_ff.replace('FAILED', np.nan, regex=True)

# df_tf["tf-num-corners"] = df_tf.count(axis=1)-1
# df_tf["gt-num-corners"] = df_gt.count(axis=1)-1
# df_ff["ff-num-corners"] = df_ff.count(axis=1)-1
# df_ff["gt-num-corners"] = df_tf["gt-num-corners"]
# df_gt["gt-num-corners"] = df_tf["gt-num-corners"]
# df_gt["tf-num-corners"] = df_tf["tf-num-corners"]
# df_gt["ff-num-corners"] = df_ff["ff-num-corners"]

In [38]:
# Make GTs on same scale as predictions
scaled_gt = df_gt.copy()

for i in range(len(scaled_gt)):
    mult = df_tf.loc[i, ["width"]] / scaled_gt.loc[i, ["width"]]
    mult = mult["width"]
    
    scaled_gt.loc[i, ["height"]] = scaled_gt.loc[i, ["height"]] * mult
    scaled_gt.loc[i, ["width"]] = scaled_gt.loc[i, ["width"]] * mult
    scaled_gt.loc[i, ["corner-1"]] = scaled_gt.loc[i, ["corner-1"]] * mult
    scaled_gt.loc[i, ["corner-2"]] = scaled_gt.loc[i, ["corner-2"]] * mult
    scaled_gt.loc[i, ["corner-3"]] = scaled_gt.loc[i, ["corner-3"]] * mult
    scaled_gt.loc[i, ["corner-4"]] = scaled_gt.loc[i, ["corner-4"]] * mult
    scaled_gt.loc[i, ["corner-5"]] = scaled_gt.loc[i, ["corner-5"]] * mult

scaled_gt = scaled_gt.round(0)

In [76]:
# # Only look at rows where num of corners is the same
# df_tf = df_tf.drop(df_tf[df_tf["tf-num-corners"] != df_tf["gt-num-corners"]].index)
# gt_tf = df_gt.drop(df_gt[df_gt["tf-num-corners"] != df_gt["gt-num-corners"]].index)
# df_ff = df_ff.drop(df_ff[df_ff["ff-num-corners"] != df_ff["gt-num-corners"]].index)
# gt_ff = df_gt.drop(df_gt[df_gt["ff-num-corners"] != df_gt["gt-num-corners"]].index)

In [40]:
def find_corners(df_gt, df_tf):
    output = []
    for i in range(df_gt.shape[0]):
        gt_np = df_gt[["corner-1", "corner-2", "corner-3", "corner-4", "corner-5"]].loc[i].dropna().to_numpy()
        tf_np = df_tf[["corner-1", "corner-2", "corner-3", "corner-4", "corner-5"]].loc[i].dropna().to_numpy()

        sub_mat = []
        for j in range(len(tf_np)):
            sub_mat.append(np.abs(gt_np - tf_np[j]))
        sub_mat = np.array(sub_mat)

        links = []
        for j in range(min(len(tf_np), len(gt_np))):
            smallest = np.amin(sub_mat)
            ind_0, ind_1 = np.where(sub_mat == smallest)
            links.append([ind_0[0], ind_1[0]])
            sub_mat[:, ind_1[0]] = 1000000
        output.append(links)

    return np.array(output)

In [41]:
# do the below on each row of df_tf and df_ff
tf_corner_ind_links = find_corners(scaled_gt, df_tf)
ff_corner_ind_links = find_corners(scaled_gt, df_ff)

c:\Users\Kim\anaconda3\envs\analysis\lib\site-packages\ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [42]:
# # Use inds to create new lists of corners in same order as GT
# new_tf = pd.DataFrame(data=[], columns=["filename", "height", "width", "corner-1", "corner-2", "corner-3", "corner-4", "corner-5"])
# new_tf[["filename", "height", "width"]] = df_tf[["filename", "height", "width"]]

# for i in range(len(new_tf)):
#     vals = np.empty(5)
#     vals[:] = np.nan
#     for j in range(len(tf_corner_ind_links[i])):
#         tf_val = df_tf.loc[i, f"corner-{tf_corner_ind_links[i][j][0]+1}"]
#         vals[tf_corner_ind_links[i][j][1]] = tf_val

#     new_tf.loc[i, ["corner-1", "corner-2", "corner-3", "corner-4", "corner-5"]] = vals

In [43]:
# Use inds to create new lists of corners in same order as GT
def get_ordered_preds(old, corner_ind_links):
    new = pd.DataFrame(data=[], columns=["filename", "height", "width", "corner-1", "corner-2", "corner-3", "corner-4", "corner-5"])
    new[["filename", "height", "width"]] = old[["filename", "height", "width"]]

    for i in range(len(new)):
        vals = np.empty(5)
        vals[:] = np.nan
        for j in range(len(corner_ind_links[i])):
            pred_val = old.loc[i, f"corner-{corner_ind_links[i][j][0]+1}"]
            vals[corner_ind_links[i][j][1]] = pred_val

        new.loc[i, ["corner-1", "corner-2", "corner-3", "corner-4", "corner-5"]] = vals

    return new

In [44]:
new_tf = get_ordered_preds(df_tf, tf_corner_ind_links)
new_ff = get_ordered_preds(df_ff, ff_corner_ind_links)

In [50]:
# Calc error for each corner
tf_err = new_tf[["filename"]]
ff_err = new_ff[["filename"]]

for i in range(5):
    tf = abs(new_tf[[f"corner-{i+1}"]] - scaled_gt[[f"corner-{i+1}"]])
    tf_err = tf_err.assign(**tf)

    ff = abs(new_ff[[f"corner-{i+1}"]] - scaled_gt[[f"corner-{i+1}"]])
    ff_err = ff_err.assign(**ff)

In [51]:
for i in range(len(tf_err)):
    for j in range(5):
        # if tf_err.loc[i, [f"corner-{j+1}"]].values[0] != np.nan:
        # print(tf_err.loc[i, [f"corner-{j+1}"]].values[0])
        tf_err.at[i, [f"corner-{j+1}"]] = (tf_err.loc[i, [f"corner-{j+1}"]].values[0] / scaled_gt.loc[i, ["width"]].values[0]) * 100
        ff_err.at[i, [f"corner-{j+1}"]] = (ff_err.loc[i, [f"corner-{j+1}"]].values[0] / scaled_gt.loc[i, ["width"]].values[0]) * 100


In [52]:
tf_err_list = np.concatenate((tf_err["corner-1"], tf_err["corner-2"],
                            tf_err["corner-3"], tf_err["corner-4"],
                            tf_err["corner-5"]))
tf_err_list = tf_err_list[~np.isnan(tf_err_list.astype(float))]

ff_err_list = np.concatenate((ff_err["corner-1"], ff_err["corner-2"],
                            ff_err["corner-3"], ff_err["corner-4"],
                            ff_err["corner-5"]))
ff_err_list = ff_err_list[~np.isnan(ff_err_list.astype(float))]

In [53]:
tf_mean_err = np.mean(tf_err_list)
ff_mean_err = np.mean(ff_err_list)

print(f"tf_mean_err = {tf_mean_err}")
print(f"ff_mean_err = {ff_mean_err}")

tf_mean_err = 13.560248183732005
ff_mean_err = 15.264777677965101


### Error of corner-floor intersection points

In [ ]:
df_ft = df_ft_input[["filename", "co-fl-in-1", "co-fl-in-2", "co-fl-in-3", "co-fl-in-4", "co-fl-in-5"]]
df_gt = df_gt_input[["filename", "co-fl-in-1", "co-fl-in-2", "co-fl-in-3", "co-fl-in-4", "co-fl-in-5"]]

df_ft = df_ft.replace('FAILED', np.nan, regex=True)

In [ ]:
# calculate error

# 30 pixels

### Compare wall coverage percentage

In [ ]:
df_tf = df_tf_input[["filename", "wall-coverage-perc"]]
df_ft = df_ft_input[["filename", "wall-coverage-perc"]]
df_ff = df_ff_input[["filename", "wall-coverage-perc"]]

df_tf = df_tf.replace('FAILED', 0, regex=True)
df_ft = df_ft.replace('FAILED', 0, regex=True)
df_ff = df_ff.replace('FAILED', 0, regex=True)

In [ ]:
df_tf["wall-coverage-perc"] = df_tf["wall-coverage-perc"].astype(float)
df_ft["wall-coverage-perc"] = df_ft["wall-coverage-perc"].astype(float)
df_ff["wall-coverage-perc"] = df_ff["wall-coverage-perc"].astype(float)

In [ ]:
# discount those where plain wall mask is used by removing 1s
df_tf = df_tf.drop(df_tf[df_tf['wall-coverage-perc'] == float(1)].index)
df_ft = df_ft.drop(df_ft[df_ft['wall-coverage-perc'] == float(1)].index)
df_ff = df_ff.drop(df_ff[df_ff['wall-coverage-perc'] == float(1)].index)

In [ ]:
tf_mean = df_tf[["wall-coverage-perc"]].mean(numeric_only=True)[0]
ft_mean = df_ft[["wall-coverage-perc"]].mean(numeric_only=True)[0]
ff_mean = df_ff[["wall-coverage-perc"]].mean(numeric_only=True)[0]

print(f"tf_mean = {tf_mean}")
print(f"ft_mean = {ft_mean}")
print(f"ff_mean = {ff_mean}")

tf_mean = 0.9137844023469459
ft_mean = 0.8236358285052049
ff_mean = 0.9166521378509982


In [ ]:
res_1 = mannwhitneyu(df_tf["wall-coverage-perc"], df_ft["wall-coverage-perc"])
res_2 = mannwhitneyu(df_tf["wall-coverage-perc"], df_ff["wall-coverage-perc"])
res_3 = mannwhitneyu(df_ft["wall-coverage-perc"], df_ff["wall-coverage-perc"])

print(f"res_1 = {res_1}")
print(f"res_2 = {res_2}")
print(f"res_3 = {res_3}")

res_1 = MannwhitneyuResult(statistic=1854.5, pvalue=0.4767893960305616)
res_2 = MannwhitneyuResult(statistic=1550.0, pvalue=0.5517142360306657)
res_3 = MannwhitneyuResult(statistic=1173.5, pvalue=0.248407364344584)
